## Fake News Classification using RNN LSTM

In [4]:
import pandas as pd

In [5]:
df=pd.read_csv(r'/content/train.csv')
#df=pd.read_csv(r'/content/test.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
#Drop Nan Values
df=df.dropna()

In [9]:
#Indepedendent Feature
X=df.drop('label',axis=1)
#Dependent Feature
y=df['label']

In [10]:
X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [12]:
messages=X.copy()

In [13]:
messages.reset_index(inplace=True)

In [14]:
#importing libarries related to word pre-processing
import nltk
import re 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Data Pre-Processing

In [16]:
### Dataset Preprocessing
ps = PorterStemmer()
corpus = []
for index,value in messages.iterrows():
    review = re.sub('[^A-z]', ' ', value['title'])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [17]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic [photos]',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hi

## One Hot Encoding

In [18]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [19]:
### Vocabulary size
voc_size=5000

In [20]:
onehot_repr=[one_hot(words,voc_size) for words in corpus] 
onehot_repr

[[2318, 4951, 695, 3367, 237, 4385, 2728, 1872, 1052, 448],
 [598, 3737, 372, 662, 288, 2000, 2124],
 [940, 2259, 4650, 3041],
 [1360, 2152, 3494, 3764, 368, 587],
 [1336, 288, 2190, 4695, 2110, 445, 288, 3700, 2631, 4105],
 [168,
  584,
  415,
  4216,
  245,
  4803,
  2060,
  3738,
  528,
  1362,
  2343,
  1371,
  624,
  4985,
  2124],
 [896, 3230, 3713, 2700, 2366, 1517, 1970, 390, 4172, 3870, 407],
 [940, 4620, 3554, 790, 2405, 4900, 4803, 4669, 4172, 3870, 407],
 [504, 2455, 2764, 3476, 2779, 3755, 3118, 2616, 4803, 1277],
 [807, 537, 1701, 3164, 170, 4777, 1957, 4429],
 [1073, 12, 937, 1562, 1473, 4823, 4027, 3752, 2080, 4773, 4436],
 [3764, 2927, 237, 3755, 4803, 2405],
 [2540, 1678, 2206, 3269, 672, 4423, 1543, 4483, 699],
 [1866, 3407, 4092, 1330, 3880, 4939, 185, 4172, 3870, 407],
 [492, 838, 876, 314, 1804, 4172, 3870, 407],
 [1415, 959, 468, 1370, 3908, 2767, 2433, 1622, 3628, 1736],
 [4020, 1005, 3737],
 [4491, 69, 424, 3197, 4803, 2987, 4839, 2124],
 [3738, 4379, 372, 1987

## Word Embedding

In [21]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1872 1052  448]
 [   0    0    0 ...  288 2000 2124]
 [   0    0    0 ... 2259 4650 3041]
 ...
 [   0    0    0 ... 4172 3870  407]
 [   0    0    0 ... 1248 1357   81]
 [   0    0    0 ... 3456  484 4594]]


In [22]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2318,
       4951,  695, 3367,  237, 4385, 2728, 1872, 1052,  448], dtype=int32)

## Model Creation

In [23]:
# Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100)) #100 is the number of Neuron
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
len(embedded_docs),y.shape

(18285, (18285,))

In [25]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

## Model Training

In [26]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [27]:
# spliting validation size at 30%
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)

In [28]:
#Final Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 7s 26ms/step - loss: 0.4813 - accuracy: 0.7499 - val_loss: 0.1976 - val_accuracy: 0.9152
Epoch 2/10
200/200 [==============================] - 5s 23ms/step - loss: 0.1470 - accuracy: 0.9422 - val_loss: 0.1969 - val_accuracy: 0.9185
Epoch 3/10
200/200 [==============================] - 5s 23ms/step - loss: 0.0974 - accuracy: 0.9651 - val_loss: 0.2159 - val_accuracy: 0.9151
Epoch 4/10
200/200 [==============================] - 5s 23ms/step - loss: 0.0645 - accuracy: 0.9777 - val_loss: 0.2539 - val_accuracy: 0.9169
Epoch 5/10
200/200 [==============================] - 5s 23ms/step - loss: 0.0506 - accuracy: 0.9838 - val_loss: 0.3029 - val_accuracy: 0.9125
Epoch 6/10
200/200 [==============================] - 5s 23ms/step - loss: 0.0381 - accuracy: 0.9892 - val_loss: 0.3622 - val_accuracy: 0.9125
Epoch 7/10
200/200 [==============================] - 5s 23ms/step - loss: 0.0268 - accuracy: 0.9938 - val_loss: 0.3738 - val_accuracy: 0.9090

In [29]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [30]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
confusion_matrix(y_pred,y_test)

array([[2799,  166],
       [ 308, 2213]])

In [31]:
accuracy_score(y_pred,y_test)

0.913598250091141

In [32]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.90      0.94      0.92      2965
           1       0.93      0.88      0.90      2521

    accuracy                           0.91      5486
   macro avg       0.92      0.91      0.91      5486
weighted avg       0.91      0.91      0.91      5486



The End.